### Question-Answer using Langchain

In [23]:
####Notes####
#not good with graphs and tables in pdfs (PyMuPDFLoader better than UnstructuredPDFLoader)
#reasonably good answers for articles with text only

In [2]:
# !pip3 install langchain
# !pip install openai
# !pip3 install openai chromadb
# !pip3 install tiktoken

In [3]:
# !pip3 install unstructured
# !pip3 install pdf2image
# !pip3 install pdfminer
# !pip3 install pdfminer.six
# !pip3 install pymupdf 

In [4]:
import os
import sys
import glob
import re
from langchain.document_loaders import WebBaseLoader, UnstructuredPDFLoader, PyMuPDFLoader
from langchain.indexes import VectorstoreIndexCreator
# from langchain.document_loaders import WebBaseLoader, BasePDFLoader

#### API Key

In [32]:
os.environ["OPENAI_API_KEY"] = ""

#### Parameters

In [6]:
pdfs_path = os.path.join(os.getcwd(), '../data/pdfs')

#### Test

###### 1. Web

In [7]:
loader = WebBaseLoader("https://www.forbes.com/sites/mattnovak/2023/07/10/google-confirms-bug-sending-less-traffic-to-online-news-publishers")
index = VectorstoreIndexCreator().from_loaders([loader])

In [10]:
index.query("Can you summarise this article in less than 200 words?")

' Google has confirmed that an issue with Google News has been identified that is causing a drop in traffic to news publishers on Monday. The issue is not yet resolved and Google has not provided any further information. This serves as a reminder of the power of Google, as when it sneezes, the internet gets a cold. The article discusses how the new version of Google Search essentially plagiarizes answers, giving users little reason to click on websites for more information.'

###### 2. PDF

In [18]:
all_pdfs = glob.glob(os.path.join(pdfs_path, '*.pdf'))

###### - single document

In [24]:
#PyMuPDFLoader
try :
    del(loader, index)
except NameError:
    pass
loader = PyMuPDFLoader(
    file_path=[_ for _ in all_pdfs if '_fund' in _][0]
)
index = VectorstoreIndexCreator().from_loaders([loader])

In [26]:
index.query("What is the breakdown by country for BGF World Technology Fund A2 SGD Hedged?")

" I don't know."

In [28]:
index.query("What is the sector breakdown for BGF World Technology Fund A2 SGD Hedged?")

' Software & Services (30.15%), Semiconductor & Equip (27.83%), Tech Hardware & Equip (11.97%), Financial Services (8.48%), Media & Entertainment (8.07%), Consumer Discretionary Distribution & Retail (4.63%), Cash and/or Derivatives (2.48%), Autos & Components (2.43%), Consumer Serv. (0.78%), Transportation (0.75%), Capital Goods (0.66%), Commercial & Professional Services (0.66%), Consumer Durables (0.58%), Other (0.52%).'

In [31]:
index.query("Which companies are part of BGF World Technology Fund A2 SGD Hedged?")

' The Fund invests globally at least 70% of its total assets in the equity securities of companies whose predominant economic activity is in the technology sector.'

In [12]:
index.query("What is the price to book ratio for BGF World Technology Fund A2 SGD Hedged?")

" I don't know."

In [11]:
index.query("What is the share of tesla in BGF World Technology Fund A2 SGD Hedged?")

' 2.44%'

In [20]:
#UnstructuredPDFLoader
try :
    del(loader, index)
except NameError:
    pass
loader = UnstructuredPDFLoader(
    file_path=[_ for _ in all_pdfs if 'em_dm' in _][0], 
    mode='single'
)
index = VectorstoreIndexCreator().from_loaders([loader])

In [21]:
index.query("What is the U.S. five-year Treasury yield in 2009 based on this article?")

' The U.S. five-year Treasury yield in 2009 was 8%.'

In [22]:
index.query("Should I invest in emerging markets or developed markets?")

' According to the BlackRock Investment Institute, they favor emerging markets over developed markets on a brighter macro backdrop. They suggest getting granular and harnessing mega forces to capture structural shifts within emerging markets.'

###### - multiple documents

In [73]:
try :
    del(loader, index)
except NameError:
    pass
loader_google = UnstructuredPDFLoader(
    file_path=[_ for _ in all_pdfs if 'google' in _][0], 
    mode='single'
)
loader_apple = UnstructuredPDFLoader(
    file_path=[_ for _ in all_pdfs if 'apple' in _][0], 
    mode='single'
)
index = VectorstoreIndexCreator().from_loaders(
    [loader_google, loader_apple]
)

In [74]:
index.query("Should I invest in google stock or apple stock?")

' It depends on your investment goals and risk tolerance. Google stock has been performing well due to its investments in artificial intelligence, while Apple has been successful in the smartphone and smart-home appliance markets. Consider researching both stocks further to determine which one is a better fit for your portfolio.'

In [77]:
index.query("What are the advantages of investing in apple stock over google stock?")

' Apple stock has an IBD Accumulation/Distribution Rating of B, indicating institutional buying of shares, and it has been trading above its 50-day and 200-day moving average lines. Google stock has shrugged off three fines totaling $9.3 billion levied by the European Union on antitrust grounds, and it has rolled out the Performance Max advertising platform.'

In [78]:
index.query("What composite rating do the best growth stocks have?")

' The best growth stocks have a Composite Rating of 90 or better.'

In [80]:
index.query("What are the biggest threats to the google stock?")

' The biggest threats to Google stock are competition in artificial intelligence and internet search, as well as investor appetite for AI stocks potentially driving a market melt-up.'

In [ ]:
###########